In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy

from utils import solr
from utils.logger import logger as log

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

In [2]:
df = pd.read_pickle('./data/gene_allele_all.pkl')

df = df.apply(lambda x: x.str.strip().str.lower())

df = df.drop(['zygosity'], axis=1)

df = df.drop_duplicates()

df_gene_allele_all = df.copy()

impc_genes = df['gene_symbol'].drop_duplicates().str.strip().str.lower()

impc_genes.shape

(8416,)

In [3]:
with open('./data/uniprot-compressed_true_download_true_format_json_query__28_28taxono-2023.02.05-17.45.13.08.json', 'r') as f:
    data = json.load(f)

In [4]:
records = []
results = data['results']
for result in results:
    primary_accession = result['primaryAccession']
    uniprotkb_id = result['uniProtkbId']
    if 'sequence' in result:
        sequence = result['sequence']['value']
        if 'genes' in result and len(result['genes']) != 0:
            for gene in result['genes']:
                gene_names = []
                if 'geneName' in gene:
                    gene_names.append(gene['geneName'])
                if 'synonyms' in gene:
                    gene_names.extend(gene['synonyms'])
                if 'orfNames' in gene:
                    gene_names.extend(gene['orfNames'])
                for gene_name in gene_names:
                    if 'value' in gene_name:
                        gene_symbol = gene_name['value']
                        records.append({
                            'gene_symbol': gene_symbol,
                            'primary_accession': primary_accession,
                            'uniprotkb_id': uniprotkb_id,
                            'sequence': sequence
                        })

In [5]:
df = pd.DataFrame(records)

df.to_pickle('./data/gene_symbol_protein_sequence_all.pkl')

_df_gene_symbol_protein_sequence_all = df.copy()

In [6]:
df = _df_gene_symbol_protein_sequence_all.copy()

df

,gene_symbol,primary_accession,uniprotkb_id,sequence
0,Hsf1,A0A075F5C6,A0A075F5C6_MOUSE,MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHVFDQGQFAKEVLPKYFKHNNMASFVRQLNMYGFRKVVHIEQGGLVKPERDDTEFQHPCFLRGQEQLLENIKRKVTSVSTLKSEDIKIRQDSVTRLLTDVQLMKGKQECMDSKLLAMKHENEALWREVASLRQKHAQQQKVVNKLIQFLISLVQSNRILGVKRKIPLMLSDSNSAHSVPKYGRQYSLEHVHGPGPYSAPSPAYSSSSLYSSDAVTSSGPIISDITELAPTSPLASPGRSIDERPLSSSTLVRVKQEPPSPPHSPRVLEASPGRPSSMDTPLSPTAFIDSILRESEPTPAASNTAPMDTTGAQAPALPTPSTPEKCLSVACLDNLARAPQMSGVARLFPCPSSFLHGRVQPGNELSDHLDAMDSNLDNLQTMLTSHGFSVDTSALLDIQELLSPQEPPRPIEAENSNPDSGKQLVHYTAQPLFLLDPDAVDTGSSELPVLFELGESSYFSEGDDYTDDPTISLLTGTEPHKAKDPTVS
1,Auts2,A0A087WPF7,AUTS2_MOUSE,MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAPSLASSSGSDKEDNGKPPSSAPSRPRPPRRKRRESTSAEEDIIDGFAMTSFVTFEALEKDVAVKPQERAEKRQTPLTKKKREALTNGLSFHSKKSRLSHSHHYSSDRENDRNLCQHLGKRKKMPKGLRQLKPGQNSCRDSDSESASGESKGFQRSSSRERLSDSSAPSSLGTGYFCDSDSDQEEKASDASSEKLFNTVLVNKDPELGVGALPEHNQDAGPIVPKISGLERSQEKSQDCCKEPVFEPVVLKDPHPQLPQLPSQAQAEPQLQIPSPGPDLVPRTEAPPQFPPPSTQPAQGPPEAQLQPAPLPQVQQRPPRPQSPSHLLQQTLPPVQSHPSSQSLSQPLSAYNSSSLSLNSLSSRSSTPAKTQPAPPHISHHPSASPFPLSLPNHSPLHSFTPTLQPPAHSHHPNMFAPPTALPPPPPLTSGSLQVPGHPAGSTYSEQDILRQELNTRFLASQSADRGASLGPPPYLRTEFHQHQHQHQHTHQHTHQHTFTPFPHAIPPTAIMPTPAPPMFDKYPTKVDPFYRHSLFHSYPPAVSGIPPMIPPTGPFGSLQGAFQPKTSNPIDVAARPGTVPHTLLQKDPRLTDPFRPMLRKPGKWCAMHVHIAWQIYHHQQKVKKQMQSDPHKLDFGLKPEFLSRPPGPSLFGAIHHPHDLARPSTLFSAAGAAHPTGTPFGPPPHHSNFLNPAAHLEPFNRPSTFTGLAAVGGNAFGGLGNPSVTPNSVFGHKDSPSVQNFSNPHEPWNRLHRTPPSFPTPPPWLKPGELERSASAAAHDRDRDVDKRDSSVSKDDKERESVEKRHPSHPSPAPPVPVSALGHNRSSTDPTTRGHLNTEAREKDKPKEKERDHSGSRKDLTTEEHKAKESHLPERDGHSHEGRAAGEEPKQLSRVPSPYVRTPGVDSTRPNSTSSREAEPRKGEPAYENPKKNAEVKVKEERKEDHDLPTEAPQAHRTSEAPPPSSSASASVHPGPLASMPMTVGVTGIHAMNSIGSLDRTRMVTPFMGLSPIPGGERFPYPSFHWDPMRDPLRDPYRDLDMHRRDPLGRDFLLRNDPLHRLSTPRLYEADRSFRDREPHDYSHHHHHHHHPLAVDPRREHERGGHLDERERLHVLREDYEHPRLHPVHPASLDGHLPHPSLLTPGLPSMHYPRISPTAGHQNGLLNKTPPTAALSAPPPLISTLGGRPGSPRRTTPLSAEIRERPPSHTLKDIEAR
2,Kiaa0442,A0A087WPF7,AUTS2_MOUSE,MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAPSLASSSGSDKEDNGKPPSSAPSRPRPPRRKRRESTSAEEDIIDGFAMTSFVTFEALEKDVAVKPQERAEKRQTPLTKKKREALTNGLSFHSKKSRLSHSHHYSSDRENDRNLCQHLGKRKKMPKGLRQLKPGQNSCRDSDSESASGESKGFQRSSSRERLSDSSAPSSLGTGYFCDSDSDQEEKASDASSEKLFNTVLVNKDPELGVGALPEHNQDAGPIVPKISGLERSQEKSQDCCKEPVFEPVVLKDPHPQLPQLPSQAQAEPQLQIPSPGPDLVPRTEAPPQFPPPSTQPAQGPPEAQLQPAPLPQVQQRPPRPQSPSHLLQQTLPPVQSHPSSQSLSQPLSAYNSSSLSLNSLSSRSSTPAKTQPAPPHISHHPSASPFPLSLPNHSPLHSFTPTLQPPAHSHHPNMFAPPTALPPPPPLTSGSLQVPGHPAGSTYSEQDILRQELNTRFLASQSADRGASLGPPPYLRTEFHQHQHQHQHTHQHTHQHTFTPFPHAIPPTAIMPTPAPPMFDKYPTKVDPFYRHSLFHSYPPAVSGIPPMIPPTGPFGSLQGAFQPKTSNPIDVAARPGTVPHTLLQKDPRLTDPFRPMLRKPGKWCAMHVHIAWQIYHHQQKVKKQMQSDPHKLDFGLKPEFLSRPPGPSLFGAIHHPHDLARPSTLFSAAGAAHPTGTPFGPPPHHSNFLNPAAHLEPFNRPSTFTGLAAVGGNAFGGLGNPSVTPNSVFGHKDSPSVQNFSNPHEPWNRLHRTPPSFPTPPPWLKPGELERSASAAAHDRDRDVDKRDSSVSKDDKERESVEKRHPSHPSPAPPVPVSALGHNRSSTDPTTRGHLNTEAREKDKPKEKERDHSGSRKDLTTEEHKAKESHLPERDGHSHEGRAAGEEPKQLSRVPSPYVRTPGVDSTRPNSTSSREAEPRKGEPAYENPKKNAEVKVKEERKEDHDLPTEAPQAHRTSEAPPPSSSASASVHPGPLASMPMTVGVTGIHAMNSIGSLDRTRMVTPFMGLSPIPGGERFPYPSFHWDPMRDPLRDPYRDLDMHRRDPLGRDFLLRNDPLHRLSTPRLYEADRSFRDREPHDYSHHHHHHHHPLAVDPRREHERGGHLDERERLHVLREDYEHPRLHPVHPASLDGHLPHPSLLTPGLPSMHYPRISPTAGHQNGLLNKTPPTAALSAPPPLISTLGGRPGSPRRTTPLSAEIRERPPSHTLKDIEAR
3,Ptgs2,A0A087WPT2,A0A087WPT2_MOUSE,MSTGFDQYKCDCTRTGFYGENCTTPEFLTRIKLLLKPTPNTVHYILTHFKGVWNIVNNIPFL
4,Fat1,A0A087WPU4,A0A087WPU4_MOUSE,XRPYFDSKLNKNIYSDIPPQVPVRPISYTPSIPSDSRNNLDRNSFEGSAIPEHPEFSTFNPESMHGHRKAVAVCSVAPNLPPPPPSNSPSDSDSIQKPSWDFDYDAKVVDLDPCLSKKPLEEKPSQPYSARESLSEVQSLSSFQSESCDDNAPIWIRAF
5,Gm20905,A0A087WRK1,A0A087WRK1_MOUSE,MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFDKNENISPQAEADEDMGDEVDSMLDKSEVNNPAIGKDENISPQVKGDEDMGHEVGSMLDKSGDDIYKTLHIKRKWMETYVKESFKGSNQKLERFCKTNERERKNINNKFCEQYITTFQKSDMDVQKFNEEKEKSVNSCQKEQQALKLSKCSQNQTLEAVKEMHEKSMEVLMNLGTKN
6,Gm20814,A0A087WRK1,A0A087WRK1_MOUSE,MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFDKNENISPQAEADEDMGDEVDSMLDKSEVNNPAIGKDENISPQVKGDEDMGHEVGSMLDKSGDDIYKTLHIKRKWMETYVKESFKGSNQKLERFCKTNERERKNINNKFCEQYITTFQKSDMDVQKFNEEKEKSVNSCQKEQQALKLSKCSQN